In [1]:
import glob
import csv
import re 
from bs4 import BeautifulSoup
import pandas as pd


* **Administration**
* **Date**
* **Note**
* **Categories**
* **Locations**
* **Names**
* **Subjects**
* **DCPD Number**


In [2]:
file_name =  './letter_sample_xml_files/address/address1.xml'
with open(file_name, "r") as f:
    text = f.read()


## Administration --> Only Borrowed From Pooja

In [3]:
presidents = ['"GWB"', '"BHO"', '"DJT"', '"WJC"']
name = ['George W. Bush', 'Barack H. Obama', 'Donald J. Trump', 'William J. Clinton']
parties = ['"R"', '"D"' ]
party_name = ['Republican', 'Democrat']

def president(txt):
    pres = re.findall('(?<=president id=).*\w\S', txt)
    x = pres[0].split()
    for president in presidents: 
        if president in x: 
            return name[presidents.index(president)]
president(text)

'Donald J. Trump'

## Date

In [4]:
def date(txt):
    date_issued = re.findall(r'<dateIssued .*>(.+?)<\/dateIssued>',txt)
    return date_issued[0]
date(text)

'2018-01-04'

## Note 

In [5]:
def note(txt):
    #notes = re.findall(r'<note>(.+?)<\/note>',txt)
    #notes2 =re.sub("&quot;","'", str(notes)) 
    soup = BeautifulSoup(txt, 'xml')
    tags = soup.find('notes')
    soup2 = BeautifulSoup(str(tags), 'xml')
    return soup2.text
note(text)

'The President spoke at 11:34 a.m. in the Roosevelt Room at the White House. In his remarks, he referred to Stephen "Michael" Garland, a U.S. Customs and Border Patrol agent who was injured in an ambush attack in Van Horn, TX, on November 18; White House Chief of Staff John F. Kelly, in his former capacity as Secretary of Homeland Security. Sen. Graham referred to former Presidents Barrack Obama and George W. Bush.'

## Categories 

In [6]:
def categories(txt): 
    soup = BeautifulSoup(txt, 'xml')
    tags = [a for a in (td.find('dcpdCategory') for td in soup.findAll('extension')) if a]
    if (tags == []):
        tags = [a for a in (td.find('category1') for td in soup.findAll('extension')) if a]
        tags = BeautifulSoup(str(tags[0]),'xml')
        return  re.sub('\n',' ',tags.text)
    else:
        tags = BeautifulSoup(str(tags[0]),'xml')
        return re.sub('\n',' ', tags.text)

categories(text)

' Addresses and Remarks Immigration reform, meeting with Republican Senators '

## Locations 

In [7]:
def location(txt): 
    soup = BeautifulSoup(text, 'xml')
    tags = [a for a in (td.find('name') for td in soup.findAll('location')) if a]
    tagss = str(tags[0])
    tagsss = re.sub('<.?name>','',tagss)
    return tagsss
location(text)

'Washington, DC'

## Names

In [8]:
def names(txt):
    soup = BeautifulSoup(txt, 'xml')
    tags = [a for a in (td.findAll('namePart') for td in soup.findAll('subject')) if a] 
    flatten_list = [x for y in tags for x in y]
    tags2 = [BeautifulSoup(str(i), "xml") for i in flatten_list]
    tags2
    no_tags = [i.text for i in tags2]
    return no_tags

names(text)

['Bannon, Stephen K.',
 'Cornyn, John, III',
 'Cotton, Thomas B.',
 'Garland, Stephen "Michael"',
 'Graham, Lindsey O.',
 'Grassley, Charles E.',
 'Kelly, John F.',
 'Lankford, James P.',
 'Nielsen, Kirstjen M.',
 'Pence, Michael R.',
 'Perdue, David A., Jr.',
 'Tillis, Thomas R.']

## Subjects

In [9]:
def subjects(txt):
    soup = BeautifulSoup(txt, 'xml')
    tags = [a for a in (td.findAll('topic') for td in soup.findAll('subject')) if a] 
    flatten_list = [x for y in tags for x in y]
    tags2 = [BeautifulSoup(str(i), "xml") for i in flatten_list]
    tags2
    no_tags = [i.text for i in tags2]
    return no_tags

subjects(text)

['Congress:Bipartisanship',
 'Congress:Members, meetings with President',
 'Drug abuse and trafficking:Illegal drugs, interdiction efforts',
 'Drug abuse and trafficking:Opioid epidemic, efforts to combat',
 'Economy, national:Improvement',
 'Homeland Security, Department of:Customs and Border Protection, U.S',
 'Homeland Security, Department of:Immigration and Customs Enforcement, U.S',
 'Homeland Security, Department of:Secretary',
 'Immigration and naturalization:Deferred Action for Childhood Arrivals (DACA) policy, review',
 'Immigration and naturalization:Illegal immigration',
 'Immigration and naturalization:Reform',
 'Legislation, enacted:Tax reform legislation',
 'Mexico:Border with U.S., infrastructure and security',
 'New York:New York City::Terrorist attacks',
 'White House Office:Assistants to the President::Chief of Staff',
 'White House Office:Vice President']

## DCPD 

In [10]:
def dcpd(txt) :
    dcpd = re.findall(r'<accessId>(.+?)</accessId>',txt)#re.findall(r'Administration of .*',text)
    dcpd2 =  re.sub("\D", "", dcpd[0])
    return dcpd2
dcpd(text)

'201800006'

# Convert list of values into CSV Format

In [11]:
president_name = []
dates  = [] 
notes = []
category = []
locations = []
list_names = []
subject = []
dcpds = []

files = glob.glob('./letter_sample_xml_files/address/*.xml')
header = ["President Name", "Date", "Notes","Categories", "Location", "Names","Subjects", "DPCD #"]

for fle in files:
 ###### open the file and then call .read() to get the text 
    with open(fle) as f:
        txt = f.read()
        president_name.append(president(txt))
        dates.append(date(txt))
        notes.append(note(txt))
        category.append(categories(txt))
        locations.append(location(txt))
        list_names.extend(names(txt))
        subject.extend(subjects(txt))
        dcpds.append(dcpd(txt))

rows = zip(president_name,dates,notes,category,locations,list_names,subject,dcpds)
with open('./letter_sample_xml_files/address/address1.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

df= pd.read_csv('./letter_sample_xml_files/address/address1.csv',names=header)


In [12]:
df

,President Name,Date,Notes,Categories,Location,Names,Subjects,DPCD #
0,Donald J. Trump,2018-01-04,The President spoke at 11:34 a.m. in the Roose...,"Addresses and Remarks Immigration reform, mee...","Washington, DC","Bannon, Stephen K.",Congress:Bipartisanship,201800006
1,Barack H. Obama,2014-01-04,NaN,Addresses and Remarks Weekly addresses,"Washington, DC","Cornyn, John, III","Congress:Members, meetings with President",201400002
2,George W. Bush,2009-01-12,The address was recorded at 10 a.m. on January...,Addresses and Remarks,"Washington, DC","Cotton, Thomas B.","Drug abuse and trafficking:Illegal drugs, inte...",200901121
3,William J. Clinton,1994-01-10,The address was recorded at 3:15 p.m. on Decem...,Addresses and Remarks,"Washington, DC","Garland, Stephen ""Michael""","Drug abuse and trafficking:Opioid epidemic, ef...",1994011012
